In [42]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import matplotlib.pyplot as plt
plt.matplotlib.rcParams.update({'font.size': 18, 'font.family': 'serif'})

import datetime
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

from utils import log_in_json, get_summary_df
from net import Net, NetCustom
from net import train_epoch, test_epoch, test_per_class

%load_ext tensorboard
%load_ext autoreload
%autoreload 2
%matplotlib inline
%tensorboard --logdir logs/tensorboard 

path_data = "./data"

## Pipeline

In [34]:
use_cuda = True
batch_size = 64
LR = 0.016
# LR = 0.001
n_epochs = 10

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

device = 'cpu' if not (use_cuda and torch.cuda.is_available()) else 'cuda'
print('Using', device)

net = NetCustom().to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=LR, momentum=0.9, dampening=0, weight_decay=0, nesterov=False)
optimizer = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=0, amsgrad=False)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

Using cuda


In [35]:
trainset = torchvision.datasets.CIFAR10(root=path_data, train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=path_data, train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

current_time = "_".join([str(eval("datetime.datetime.now().time().{:0>2}".format(x))) for x in ["hour", "minute", "second"]])
print('Logging into logs/tensorboard/{}'.format(current_time))
path_log = 'logs/tensorboard/' + current_time

writer = SummaryWriter(path_log)

globaliter = 0
for epoch in range(1, n_epochs + 1):
    globaliter = train_epoch(trainloader, net, criterion, optimizer, scheduler, epoch,
                             device, 500, globaliter, writer)
    test_epoch(testloader, net, criterion, epoch, device, writer)

Logging into logs/tensorboard/18_40_11
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.309652
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.443730
Test Accuracy: 64.53%
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.018696
Train Epoch: 2 [32000/50000 (64%)]	Loss: 0.949532
Test Accuracy: 69.40%
Train Epoch: 3 [0/50000 (0%)]	Loss: 0.699478
Train Epoch: 3 [32000/50000 (64%)]	Loss: 0.704379
Test Accuracy: 71.25%
Train Epoch: 4 [0/50000 (0%)]	Loss: 0.573306
Train Epoch: 4 [32000/50000 (64%)]	Loss: 0.587019
Test Accuracy: 71.79%
Train Epoch: 5 [0/50000 (0%)]	Loss: 0.443250
Train Epoch: 5 [32000/50000 (64%)]	Loss: 0.394189
Test Accuracy: 74.44%
Train Epoch: 6 [0/50000 (0%)]	Loss: 0.299663
Train Epoch: 6 [32000/50000 (64%)]	Loss: 0.477990
Test Accuracy: 74.69%
Train Epoch: 7 [0/50000 (0%)]	Loss: 0.360719
Train Epoch: 7 [32000/50000 (64%)]	Loss: 0.399042
Test Accuracy: 75.05%
Train Epoch: 8 [0/50000 (0%)]	Loss: 0.395322
Train Epoch: 8 [32000/50000 (64%)]	Loss: 0.345058
Test Accuracy: 74.59%
Train Epoch: 9 [0/50000 (

In [36]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

test_accuracy = test_epoch(testloader, net, criterion, device=device)
test_per_class(testloader, net, classes, device=device)

log_in_json(net, criterion, batch_size, optimizer, scheduler, n_epochs, device, current_time, test_accuracy)

Test Accuracy: 74.73%
Accuracy of plane : 87 %
Accuracy of   car : 96 %
Accuracy of  bird : 64 %
Accuracy of   cat : 53 %
Accuracy of  deer : 69 %
Accuracy of   dog : 62 %
Accuracy of  frog : 76 %
Accuracy of horse : 81 %
Accuracy of  ship : 89 %
Accuracy of truck : 79 %


## Summary

In [43]:
get_summary_df()

,model,criterion,batch_size,optimizer,scheduler,n_epochs,device,test_accuracy
timestamp,,,,,,,,
18_36_27,"NetCustom( (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1)) (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)) (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (fc1): Linear(in_features=1152, out_features=128, bias=True) (fc2): Linear(in_features=128, out_features=64, bias=True) (fc3): Linear(in_features=64, out_features=10, bias=True) )",CrossEntropyLoss(),64,"SGD(lr: 0.016, momentum: 0.9, dampening: 0, weight_decay: 0, nesterov: False)","StepLR(gamma: 0.1, step_size: 5)",10,cuda,74.90%
18_40_11,"NetCustom( (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1)) (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)) (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (fc1): Linear(in_features=1152, out_features=128, bias=True) (fc2): Linear(in_features=128, out_features=64, bias=True) (fc3): Linear(in_features=64, out_features=10, bias=True) )",CrossEntropyLoss(),64,"Adam(lr: 0.001, betas: (0.9, 0.999), eps: 1e-08, weight_decay: 0, amsgrad: False)","StepLR(gamma: 0.1, step_size: 5)",10,cuda,74.73%
18_33_7,"NetCustom( (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1)) (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)) (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (fc1): Linear(in_features=1152, out_features=128, bias=True) (fc2): Linear(in_features=128, out_features=96, bias=True) (fc3): Linear(in_features=96, out_features=10, bias=True) )",CrossEntropyLoss(),64,"SGD(lr: 0.016, momentum: 0.9, dampening: 0, weight_decay: 0, nesterov: False)","StepLR(gamma: 0.1, step_size: 5)",10,cuda,74.57%
18_39_22,"NetCustom( (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1)) (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)) (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (fc1): Linear(in_features=1152, out_features=128, bias=True) (fc2): Linear(in_features=128, out_features=64, bias=True) (fc3): Linear(in_features=64, out_features=10, bias=True) )",CrossEntropyLoss(),64,"Adam(lr: 0.0008, betas: (0.9, 0.999), eps: 1e-08, weight_decay: 0, amsgrad: False)","StepLR(gamma: 0.1, step_size: 5)",10,cuda,74.36%
17_58_29,"Net( (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1)) (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1)) (fc1): Linear(in_features=400, out_features=120, bias=True) (fc2): Linear(in_features=120, out_features=84, bias=True) (fc3